# Streamlit
Streamlit lets you transform your data scripts into interactive dashboards and prototypes in minutes, without needing front-end coding knowledge. This means you can easily share insights with colleagues, showcase your data science work, or even build simple machine learning tools, all within the familiar Python environment.

---
## 1.&nbsp; Streamlit demo 🚀
We first need to install [streamlit](https://streamlit.io/) - as always, locally this is a one time thing, whereas on colab we need to do it each session.

We're also installing [localtunnel](https://theboroer.github.io/localtunnel-www/) here, which allows us to run streamlit from colab. If you're working locally, you don't need to install this.

In [ ]:
!pip install -qqq streamlit --progress-bar off
!npm install -qqq localtunnel --progress-bar off

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
+ off@0.0.10
added 23 packages from 23 contributors and audited 23 packages in 2.425s

3 packages are looking for funding
  run `npm fund` for details

found 1 moderate severity vulnerability
  run `npm audit fix` to fix them, or `npm audit` for details


Here's an example of what you can produce with streamlit. It's so easy, just a few lines of python depending on what you want, and so many options!

- Locally you would write this script in a .py file and not a notebook (.ipynb).

- On colab, we can create a .py file by using the magic command `%%writefile` at the top of the cell. This command writes the cell content to a file, naming it 'app.py', or whatever else you choose, in this instance. Once saved, you can see 'app.py' in Colab's storage by clicking on the left-hand side folder icon.

In [ ]:
%%writefile app.py

import streamlit as st

# Title
st.title("Streamlit Demo")

# Markdown
st.markdown("""
This is a demo app showcasing a few of Streamlit's features.

Streamlit is a powerful Python library for creating web apps. It is easy to use and has a wide range of features, including:

* **Interactive widgets:** Streamlit makes it easy to create interactive widgets, such as sliders, dropdown menus, and radio buttons.
* **Charts and graphs:** Streamlit can generate a variety of charts and graphs, including line charts, bar charts, and pie charts.
* **Data display:** Streamlit can display data in a variety of ways, including tables, lists, and maps.
* **Deployment:** Streamlit apps can be deployed to Heroku with a single command.
""")

# Slider
slider_value = st.slider("Select a number:", 0, 100)
st.write(f"You selected: {slider_value}")

# Dropdown menu
dropdown_value = st.selectbox("Choose a color:", ["red", "green", "blue"])
st.write(f"You chose: {dropdown_value}")

# Radio buttons
radio_button_value = st.radio("Select a language:", ["English", "Spanish", "French"])
st.write(f"You selected: {radio_button_value}")

# Text area
text = st.text_area("Enter some text:")
if text:
    st.write(f"You entered: {text}")

# Button
if st.button("Click me!"):
    st.write("You clicked the button!")

# Chart
data = {"x": [1, 2, 3, 4, 5], "y": [6, 7, 2, 4, 5]}
st.line_chart(data)

# Map
map_data = [
    {"name": "New York", "lat": 40.7128, "lon": -74.0060},
    {"name": "Los Angeles", "lat": 34.0522, "lon": -118.2437},
    {"name": "Chicago", "lat": 41.8783, "lon": -87.6233},
]
st.map(map_data)

Writing app.py


To run streamlit apps **locally**. Open the command line and navigate to the folder where you've stored the .py file. Then, use the command `streamlit run app.py`. If you used a different name for the .py file, change `app.py` for the name you used

On **colab**, as we have to run streamlit through a tunnel. This is a little annoying for debugging, as every time you encounter a bug, you have to stop and reopen the tunnel. However, if you have a slower computer, or you simply wish to use Google's power so that your resources are free to do other things, it's very useful.

When opening the tunnel it will ask you for a password. The password is your IP address, which will be printed out as the first element of the output from the code below.

In [ ]:
!streamlit run app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

34.34.23.251
npx: installed 22 in 1.704s
your url is: https://easy-memes-repeat.loca.lt


## 2.&nbsp; RAG chatbot in streamlit ⭐️
We'll start by installing the same libraries and downloading the same files as in the previous notebooks.

In [ ]:
!pip3 install -qqq langchain --progress-bar off
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip3 install -qqq llama-cpp-python --progress-bar off
!pip3 install -qqq sentence_transformers --progress-bar off
!pip3 install -qqq faiss-gpu --progress-bar off

!huggingface-cli download TheBloke/Mistral-7B-Instruct-v0.1-GGUF mistral-7b-instruct-v0.1.Q4_K_M.gguf --local-dir . --local-dir-use-symlinks False

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
Consider using `hf_transfer` for faster downloads. This solution comes with some limitations. See https://huggingface.co/docs/huggingface_hub/hf_transfer for more details.
downloading https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q4_K_M.gguf to /root/.cache/huggingface/hub/tmpuja86ou_
mistral-7b-instruct-v0.1.Q4_K_M.gguf: 100% 4.37G/4.37G [00:39<00:00, 111MB/s] 
./mistral-7b-instruct-v0.1.Q4_K_M.gguf


In [ ]:
# in case you get the error 'NoneType' object has no attribute 'groups'
# !pip install --upgrade gdown

# download saved vector database for Alice's Adventures in Wonderland
!gdown --folder 1A8A9lhcUXUKRrtCe7rckMlQtgmfLZRQH

  Attempting uninstall: gdown
    Found existing installation: gdown 4.7.3
    Uninstalling gdown-4.7.3:
      Successfully uninstalled gdown-4.7.3
Retrieving folder contents
Processing file 1h_lk4wTr12FAEaCS3eIJ4xsdcmnuIGmt index.faiss
Processing file 1O0Jz2Lx5cZdpQM7S5uw6Kx9_OLm5DuSQ index.pkl
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1h_lk4wTr12FAEaCS3eIJ4xsdcmnuIGmt
To: /content/faiss_index/index.faiss
100% 421k/421k [00:00<00:00, 47.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1O0Jz2Lx5cZdpQM7S5uw6Kx9_OLm5DuSQ
To: /content/faiss_index/index.pkl
100% 216k/216k [00:00<00:00, 102MB/s]
Download completed


Now, let's proceed by creating the .py file for our rag chatbot.

We sourced the foundational code for our Streamlit basic chatbot from the [Streamlit documentation](https://docs.streamlit.io/knowledge-base/tutorials/build-conversational-apps).

In addition, we implemented [cache_resource](https://docs.streamlit.io/library/api-reference/performance/st.cache_resource) for both memory and LLM. Given that Streamlit reruns the entire script with each message input, relying solely on memory would result in data overwriting and a loss of conversational continuity. The inclusion of cache resource prevents Streamlit from creating a new memory instance on each run. This was also added to the LLM, enhancing speed and preventing its reload in every iteration.

In [ ]:
%%writefile rag_app.py

from langchain.llms import LlamaCpp
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationalRetrievalChain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.memory import ConversationBufferMemory
import streamlit as st

# llm
@st.cache_resource
def init_llm():
  return LlamaCpp(model_path = "/content/mistral-7b-instruct-v0.1.Q4_K_M.gguf",
                  max_tokens = 2000,
                  temperature = 0.1,
                  top_p = 1,
                  n_gpu_layers = -1,
                  n_ctx = 1024)
llm = init_llm()

# embeddings
embedding_model = "sentence-transformers/all-MiniLM-l6-v2"
embeddings_folder = "/content/embeddings"
embeddings = HuggingFaceEmbeddings(model_name=embedding_model,
                                   cache_folder=embeddings_folder)

# load Vector Database
# allow_dangerous_deserialization is needed. Pickle files can be modified to deliver a malicious payload that results in execution of arbitrary code on your machine
vector_db = FAISS.load_local("/content/faiss_index", embeddings, allow_dangerous_deserialization=True)

# retriever
retriever = vector_db.as_retriever(search_kwargs={"k": 2})

# memory
@st.cache_resource
def init_memory(_llm):
    return ConversationBufferMemory(
        llm=llm,
        output_key='answer',
        memory_key='chat_history',
        return_messages=True)
memory = init_memory(llm)

# prompt
template = """
<s> [INST]
You are polite and professional question-answering AI assistant. You must provide a helpful response to the user.

In your response, PLEASE ALWAYS:
  (0) Be a detail-oriented reader: read the question and context and understand both before answering
  (1) Start your answer with a friendly tone, and reiterate the question so the user is sure you understood it
  (2) If the context enables you to answer the question, write a detailed, helpful, and easily understandable answer. If you can't find the answer, respond with an explanation, starting with: "I couldn't find the answer in the information I have access to".
  (3) Ensure your answer answers the question, is helpful, professional, and formatted to be easily readable.
[/INST]
[INST]
Answer the following question using the context provided.
The question is surrounded by the tags <q> </q>.
The context is surrounded by the tags <c> </c>.
<q>
{question}
</q>
<c>
{context}
</c>
[/INST]
</s>
[INST]
Helpful Answer:
[INST]
"""

prompt = PromptTemplate(template=template,
                        input_variables=["context", "question"])

# chain
chain = ConversationalRetrievalChain.from_llm(llm,
                                              retriever=retriever,
                                              memory=memory,
                                              return_source_documents=True,
                                              combine_docs_chain_kwargs={"prompt": prompt})


##### streamlit #####

st.title("Chatier & chatier: conversations in Wonderland")

# Initialise chat history
# Chat history saves the previous messages to be displayed
if "messages" not in st.session_state:
    st.session_state.messages = []

# Display chat messages from history on app rerun
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# React to user input
if prompt := st.chat_input("Curious minds wanted!"):

    # Display user message in chat message container
    st.chat_message("user").markdown(prompt)

    # Add user message to chat history
    st.session_state.messages.append({"role": "user", "content": prompt})

    # Begin spinner before answering question so it's there for the duration
    with st.spinner("Going down the rabbithole for answers..."):

        # send question to chain to get answer
        answer = chain(prompt)

        # extract answer from dictionary returned by chain
        response = answer["answer"]

        # Display chatbot response in chat message container
        with st.chat_message("assistant"):
            st.markdown(answer["answer"])

        # Add assistant response to chat history
        st.session_state.messages.append({"role": "assistant", "content": response})

Writing rag_app.py


Now, let's load a tunnel and see what we've made

In [ ]:
!streamlit run rag_app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

35.239.60.84
npx: installed 22 in 2.572s
your url is: https://shy-masks-invent.loca.lt
